In [3]:
import numpy as np #untuk analisis data, buat tabel, cek data
import pandas as pd #numerical python,membuat array

df = pd.read_excel(r"C:\Users\user\Documents\data_aspek_pos.xlsx")
df #tampilkan data frame

,opini,aspek
0,berita cepat terkini,content
1,tarik berita baru,content
2,berita bagus sesuai jadi,content
3,aplikasi henti,functionality
4,berita terkini mantap percaya maju cnn indones...,content
...,...,...
995,profesional serang bijak perintah sa,content
996,berita ber ulang ulang muncul berita banyak ik...,content
997,berita diskredit pemerintahbila berita sampai ...,content
998,baik berita terus,content


In [4]:
#jumlah per aspek
df['aspek'].value_counts() 

content              777
functionality        192
user interface        16
streaming quality     15
Name: aspek, dtype: int64

In [5]:
#TF IDF
from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer 
vectorizer = CountVectorizer(decode_error="replace")
transformer = TfidfTransformer()
TFIDF = transformer.fit_transform(vectorizer.fit_transform(df['opini']))

In [6]:
print(TFIDF)

  (0, 1516)	0.5924283618913982
  (0, 301)	0.7578853615158642
  (0, 203)	0.27320031996072486
  (1, 1479)	0.6729059408194431
  (1, 203)	0.21677723549801212
  (1, 163)	0.7072518822737338
  (2, 1394)	0.566766929490242
  (2, 641)	0.721910996369818
  (2, 203)	0.18118446935185054
  (2, 123)	0.35325903954779125
  (3, 563)	0.8986765671302706
  (3, 87)	0.4386119329098928
  (4, 1516)	0.2517695255590892
  (4, 1171)	0.32379195307252456
  (4, 916)	0.6658473804821057
  (4, 890)	0.485435984873708
  (4, 605)	0.29047295335276574
  (4, 323)	0.23434011376259567
  (4, 203)	0.11610435854134919
  (5, 1516)	0.3789111895379337
  (5, 1171)	0.48730438614075494
  (5, 1021)	0.492613439306687
  (5, 586)	0.5101228831123124
  (5, 123)	0.34068664375133567
  (6, 1084)	0.9725689930811491
  :	:
  (995, 1306)	0.508194081052707
  (995, 1216)	0.45937583676241306
  (995, 1174)	0.32232569836784586
  (995, 227)	0.508194081052707
  (996, 1606)	0.7011906987170873
  (996, 996)	0.3018701821203632
  (996, 587)	0.4015925092163663
  

In [7]:
  #Save model TFIDF
import pickle
pickle.dump(vectorizer.vocabulary_,open("TFIDF_kfold.pkl","wb"))

In [8]:
labels = [] #membuat kolom labels
for data in df['aspek']:#untuk data di df row kategori
    labels.append(data)
kolom = labels.pop
print(labels)

['content', 'content', 'content', 'functionality', 'content', 'content', 'content', 'content', 'functionality', 'content', 'user interface', 'content', 'functionality', 'functionality', 'content', 'content', 'functionality', 'streaming quality', 'content', 'content', 'content', 'content', 'content', 'content', 'content', 'content', 'content', 'streaming quality', 'content', 'content', 'functionality', 'content', 'content', 'content', 'content', 'content', 'content', 'content', 'functionality', 'functionality', 'content', 'content', 'content', 'functionality', 'content', 'content', 'content', 'functionality', 'content', 'streaming quality', 'content', 'content', 'content', 'content', 'content', 'content', 'content', 'content', 'content', 'streaming quality', 'content', 'content', 'content', 'content', 'user interface', 'content', 'content', 'content', 'content', 'content', 'streaming quality', 'content', 'content', 'content', 'content', 'content', 'functionality', 'content', 'functional

In [9]:
#kemudian kategori diberi label
label=[]
for i in range(len(labels)): 
    if labels[i]=='content':
        nilai = 0
        label.append(nilai)
    
    if labels[i]=='functionality':
        nilai = 1
        label.append(nilai)
    
    if labels[i]=='user interface':
        nilai = 2
        label.append(nilai)
        
    if labels[i]=='streaming quality':
        nilai = 3
        label.append(nilai)

In [10]:
from sklearn import svm #metode untuk klasifikasi aspek / kategori
from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer #untuk menghitung vsm dan tfidf #vsm =vector space model
from sklearn import metrics #metode untuk pembentukan matriks 1x1 1x2 1x3
from sklearn.metrics import accuracy_score #metod untuk pembentukan skor akurasi 
from sklearn.model_selection import KFold #metode untuk perhitungan kfold
import numpy as np 
import re #regular expression
import itertools #fungsi pengulangan tak terbatas
import matplotlib.pyplot as plt #pembentukan diagram
from sklearn.metrics import confusion_matrix
from sklearn.svm import SVC #suport vector classifier
from sklearn.metrics import classification_report

##packages untuk multiclass
from sklearn.multiclass import OneVsRestClassifier #pake library ovr
##from sklearn.svm import SVC
from sklearn.svm import LinearSVC

In [14]:
akurasi=[]
def avg_akurasiOvR(): ##fungsi untuk rata rata akurasi one vs rest classifier
    total = 0
    for i in range(5): ##lakukan 10 kali
        total = total + akurasi[i]
    print("Rata-rata akurasi keseluruhan SVM Multiclass One vs All adalah :", total / 5)

In [15]:
kFoldCrossValidation = KFold(n_splits=5, random_state=42, shuffle = True)#fungsi K-Fold Cross Validation melakukan insialisasi 10 kali iterasi

In [16]:
from sklearn.multiclass import OneVsRestClassifier #pake library ovr
for latih, uji in kFoldCrossValidation.split(TFIDF, label):
    
    print("-----------------------------")
    print("Banyak Data Latih: ", len(latih)) #berapa jmlah data latih
    print("Banyak Data Uji: ", len(uji)) #berapa jumlah data uji
    print("\nData Latih: \n", latih) #menampilkan data latih
    print("\nData Uji: \n", uji)#menampilkan data uji
    dataLatih1, dataUji1 = TFIDF[latih], TFIDF[uji]#proses inisialisasi dari masing2 data latih/uji dijadikan nilai tfidf lalu di copy ke variabel dataLatih/Uji1
    label = np.array(label)
    dataLatih2, dataUji2 = label[latih], label[uji]#
    
    SVMOvR = OneVsRestClassifier(LinearSVC(C=1))
    modelOvR = SVMOvR.fit(dataLatih1, dataLatih2)
    prediksi = modelOvR.predict(dataUji1)
    
    print("\nHasil Prediksi SVM One vs All : \n", prediksi)
    
    print("\nConfusion Matrix: \n", metrics.confusion_matrix(dataUji2, prediksi))
   
    akurasi.append(accuracy_score(dataUji2, prediksi))
    
    print("\nAkurasi SVM One vs All : ", accuracy_score(dataUji2, prediksi))
    print()\
    
    label_target = ['content','functionality','user interface','streaming quality']
    print(metrics.classification_report(dataUji2, prediksi, labels=[0,1,2,3]))#proses pembentukan confusin matrix
    
avg_akurasiOvR() #hasil rata rata multiclass

-----------------------------
Banyak Data Latih:  800
Banyak Data Uji:  200

Data Latih: 
 [  0   1   2   3   4   5   6   7   8   9  11  12  13  14  15  16  17  18
  19  20  21  22  24  26  27  28  29  31  32  33  34  35  36  37  38  40
  41  42  43  45  46  47  48  49  50  51  52  53  56  57  58  61  62  64
  65  68  69  71  73  74  75  77  79  80  81  82  83  84  85  87  89  90
  91  92  93  94  95  97  98  99 100 102 103 104 105 106 108 109 111 112
 113 114 115 116 117 118 119 121 122 123 124 125 126 127 128 129 130 131
 132 133 134 135 138 140 141 142 143 144 145 146 147 148 149 150 151 152
 153 154 155 156 157 159 160 161 162 163 164 165 166 167 169 170 171 172
 173 175 176 177 178 179 180 181 182 183 185 186 187 188 189 190 191 192
 193 194 195 196 197 200 201 202 203 204 205 206 207 211 212 214 216 217
 219 220 222 223 224 225 226 227 228 229 230 231 232 233 234 236 238 239
 240 241 242 243 245 246 248 249 250 251 252 253 255 257 258 262 263 264
 266 267 268 269 270 271 272 273 

C:\Users\user\AppData\Roaming\Python\Python310\site-packages\sklearn\metrics\_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\user\AppData\Roaming\Python\Python310\site-packages\sklearn\metrics\_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\user\AppData\Roaming\Python\Python310\site-packages\sklearn\metrics\_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C

# Uji model

In [17]:
# save the model to disk
import pickle
filename = 'OnevsAll.sav'
pickle.dump(SVMOvR, open(filename, 'wb'))

In [18]:
import string
import Sastrawi
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory

In [19]:
kalimat = "gak bisa kebaca temanya gelap sekali" 
lower_case = kalimat.lower()
print (lower_case)

gak bisa kebaca temanya gelap sekali


In [20]:
punctuation = lower_case.translate(str.maketrans('','',string.punctuation)).strip()
print(punctuation)

gak bisa kebaca temanya gelap sekali


In [21]:
tokenize = word_tokenize(punctuation)
print(tokenize)

['gak', 'bisa', 'kebaca', 'temanya', 'gelap', 'sekali']


In [22]:
factory = StopWordRemoverFactory()
stopword = factory.create_stop_word_remover()
hasil_stopword = []
for i in tokenize:
    word = stopword.remove(i)
    if word !='':
        hasil_stopword.append(word)
print(hasil_stopword)

['gak', 'kebaca', 'temanya', 'gelap', 'sekali']


In [23]:
factory = StemmerFactory()
stemmer = factory.create_stemmer()  
hasil_stemming = [] 
for i in hasil_stopword:
    word = stemmer.stem(i)
    if word !='':
        hasil_stemming.append(word) 
print(hasil_stemming)

['gak', 'baca', 'tema', 'gelap', 'sekali']


In [24]:
teks_ulasan=[' '.join(reversed(hasil_stemming))]
print(teks_ulasan)

['sekali gelap tema baca gak']


In [25]:
#Load TFIDF model
import pickle
from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer
transformer = TfidfTransformer()
loaded_vec = CountVectorizer(decode_error="replace",vocabulary=pickle.load(open("TFIDFmodel_kfold.pkl", "rb")))
tfidf = transformer.fit_transform(loaded_vec.fit_transform(teks_ulasan))

In [26]:
print (tfidf)

  (0, 1491)	0.5
  (0, 504)	0.5
  (0, 492)	0.5
  (0, 118)	0.5


In [27]:
loaded_SVM_model = pickle.load(open('OnevsAll.sav', 'rb'))

In [28]:
print (loaded_SVM_model)

OneVsRestClassifier(estimator=LinearSVC(C=1))


In [30]:
loaded_SVM_model.predict(tfidf) #hasil prediksi

array([2])